In [1]:
!pip install -q dotenv langchain-google-genai ollama pandas

In [4]:
import os

# os.environ["OLLAMA_HOST"] = "https://a973-34-53-30-6.ngrok-free.app"
os.environ["OLLAMA_HOST"] = "https://429d-35-247-20-120.ngrok-free.app"

In [5]:
!ollama list



Error: Head "https://429d-35-247-20-120.ngrok-free.app:443/": dial tcp: lookup 429d-35-247-20-120.ngrok-free.app: no such host


In [3]:
!ollama list

NAME           ID              SIZE      MODIFIED      
llama3.2:3b    a80c4f17acd5    2.0 GB    9 minutes ago    


In [6]:
# !ollama pull qwen2.5:7b
# !ollama pull gemma3:4b
# !ollama pull gemma3:12b
# !ollama pull phi3:14b
# !ollama pull phi3.5
# !ollama pull phi4
# !ollama pull phi4-mini
!ollama pull llama3.2:3b

!ollama list

Error: Head "https://429d-35-247-20-120.ngrok-free.app:443/": dial tcp: lookup 429d-35-247-20-120.ngrok-free.app: no such host


NAME    ID    SIZE    MODIFIED 


In [5]:
def read_file(filename):
    file = open(filename,'r')
    str = file.read()
    return str

def get_question_prompt(sql_file):
    prompt = f""" 
    ### Instructions:
    Your task is to create a text question based on given database schema.
    Adhere to these rules:
    - **Deliberately go through the database schema word by word** to appropriately create the question.
    - **Create a question that will require joins and aliases** to increase difficulty of question.
    - When creating a question, always stick to the given schema.

    ### Input:
    Generate a Text Question based on the following Database Schema `{read_file(sql_file)}`.
    
    This question will assess ability and knowledge of SQL, Database and DBMS.

    ### Response:
    Based on your instructions, here is the Text Question I have generated from the provided schema, without any additional text.:
    """
    return prompt


def get_custom_prompt(question,sql_file):
    # schema = open(sql_file,'r').read()

    prompt = f""" 
    ### Instructions:
    Your task is to convert a question into a SQL query, given a Postgres database schema.
    Adhere to these rules:
    - **Deliberately go through the question and database schema word by word** to appropriately answer the question
    - **Use Table Aliases** to prevent ambiguity. For example, `SELECT table1.col1, table2.col1 FROM table1 JOIN table2 ON table1.id = table2.id`.
    - When creating a ratio, always cast the numerator as float

    ### Input:
    Generate a SQL query that answers the question `{question}`.
    
    This query will run on a database whose schema is represented in this string:

    {read_file(sql_file)}

    ### Response:
    Based on your instructions, here is the SQL query I have generated to answer the question `{question}` without any additional text.:
    """
    return prompt

def get_rate_prompt(schema,question,answer):
    prompt = f"""

    ### Instructions:
    Your Task is to throughly study the given SQL Schema, and the text question asked based on the schema. you will be also be given a SQL query, which you have to rate on scale of 10.
    
    Adhere to these rules:
    - **Deliberately go through the question and database schema word by word** to appropriately answer the question
    - **Use Table Aliases** to prevent ambiguity. For example, `SELECT table1.col1, table2.col1 FROM table1 JOIN table2 ON table1.id = table2.id`.
    - When creating a ratio, always cast the numerator as float


    ## Input:
    A question, {question}, is asked based on schema as given below.
    
    Overall Rate the SQL query on scale of 10, `{answer}`, based on the efficiency simplicity.
    This Query is suppose to run on the schema in the following string:

    {read_file(schema)}

    I DONT WANT EXPLAINATION FOR YOUR RATINGS! I ONLY WANT THE RATING! THATS IT!

    

    ### Response:
    Based on your instructions, here is the rating of the sql query for provided schema without any additional text.:
    """

    return prompt


def get_ratings(schema,question,answer):
    response = llm_gemini.invoke(
        get_rate_prompt(
            schema = schema,
            question = question,
            answer = answer,
            )
    )
    
    return response.content

In [6]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

llm_gemini = ChatGoogleGenerativeAI(
    model = "gemini-2.0-flash",
    api_key = os.getenv("GEMINI_API_KEY")
)


In [262]:
# ans=llm_gemini.invoke("tell me a joke")
# ans.content

In [10]:
questions = [
    "List all drugs in the 'Painkillers' category with a stock below 100 units, along with their supplier names and the date when their inventory was last updated. Order results by oldest inventory update first.",
    "Identify the top 5 products by total sales amount in the last quarter of 2024, including their category names and the store locations where they were sold. Order the results by total sales amount in descending order.",
    "Find the top 3 products with the highest average rating from user reviews, including their category names and total sales amounts. Only consider products that have been ordered at least 10 times. Order the results by average rating in descending order.",
    "List all agents who have facilitated sales of properties with a total sale price exceeding $500,000 in the last year. Include their full names, the number of properties sold, and the total commission earned. Order the results by total commission in descending order.",
    "Identify the top 5 products by total sales amount in the last quarter of 2024, including the name of the employee who made the sale and the total quantity sold for each product. Order the results by total sales amount in descending order.",
    "Retrieve the first name of customers who have given a 5-star rating to a restaurant and have also placed at least one order at that restaurant. For each such customer-restaurant pair, also retrieve the name of the restaurant and the total amount the customer has spent at that restaurant.",
    "Retrieve the first and last name of customers who have purchased a vehicle and also requested a service for that same vehicle, along with the model of the vehicle and the total cost of all services for that vehicle.",
    "How can you retrieve a list of all active subscriptions along with the customer's name, plan name, and the subscription's start and end dates",
    "How can you retrieve a report showing the total stock level of each product across all warehouses, including the product name and total quantity available?",
    "How can you generate a report showing the total sales revenue for each salesperson, including their name and the total amount sold?",
]

responses = []
ratings = []

In [11]:
# dict={
#     'schema':schema,
#     'question':question,
#     'sqlanswer':sqlanswer,
#     'rating':rating,
#     'model':model,
# }

In [ ]:

# import ollama

# model_list = ['qwen2.5:7b','gemma3:4b','phi3','phi3.5']

# dict=[]

# for model in model_list:
    
#     for i in range(5):
#         FILENAME = f"table_list{i+1}_exp.sql"

#         response = ollama.chat(model=model, messages=[
#         {
#             'role': 'user',
#             # 'content': get_prompt("What are the names and prices of all drugs in the 'Antibiotics' category?")
#             'content': get_custom_prompt(
#                 question = questions[i],
#                 sql_file = FILENAME
#             )
#             # 'content': get_prompt("Which pharmacies have sold more than 100 units of drugs from the 'Painkillers' category in the last 30 days, and what is the total revenue generated from those sales, including the names of the employees who made the sales?")
#         },
#         ])

#         res = response['message']['content']
        

#         responses.append(res)

#         rating = get_ratings(
#             schema = f"table_list{i+1}_exp.sql",
#             question = questions[i],
#             answer = res,
#         )

#         ratings.append(rating)

#         d={
#             'schema':read_file(FILENAME),
#             'model':model,
#             'question':questions[i],
#             'sqlanswer':res,
#             'rating':rating
#         }

#         dict.append(d)




In [13]:
!ollama list
# !ollama pull llama3.2:3b

NAME           ID              SIZE      MODIFIED      
llama3.2:3b    a80c4f17acd5    2.0 GB    2 minutes ago    


In [ ]:
# import ollama

# # model_list = ['qwen2.5:7b','gemma3:4b','gemma3:12b','phi3:latest','phi3:14b','phi3.5:latest','phi4:latest','phi4-mini:latest']

# model_list = ['llama3.2:3b']#'qwen2.5:7b','gemma3:4b','gemma3:12b','phi3:latest','phi3:14b','phi3.5:latest','phi4:latest','phi4-mini:latest']

# dict=[]

# # textquestion_llm = llm_gemini

# for model in model_list:
#     for i in range(10):
#         FILENAME = f"table_list{i+1}_exp.sql"

#         llm_question = ollama.chat(model=model,messages=[
#             {
#                 'role':'user',
#                 'content':get_question_prompt(
#                     sql_file=FILENAME
#                 )
#             }
#         ])

#         # llm_question = llm_gemini.invoke(
#         #     input=get_question_prompt(FILENAME)
#         # )

#         response = ollama.chat(model=model, messages=[
#         {
#             'role': 'user',
#             'content': get_custom_prompt(
#                 question = llm_question,
#                 sql_file = FILENAME
#             )
#         },
#         ])

#         res = response['message']['content']
#         print(res)
#         responses.append(res)

#         rating = get_ratings(
#             schema = f"table_list{i+1}_exp.sql",
#             question = llm_question,
#             answer = res,
#         )
#         ratings.append(rating)

#         d={
#             'schema':read_file(FILENAME),
#             'model':model,
#             'question':llm_question,
#             'sqlanswer':res,
#             'rating':rating
#         }

#         dict.append(d)

        
#         print('*'*50)
#         print(d)
#         print('*'*50)

SELECT T1.employee_id, SUM(T2.total_amount) AS total_spent, COUNT(DISTINCT T3.pharmacy_name) AS num_pharmacies, SUM(T3.quantity * T4.price) / 100 AS avg_price FROM employee AS T1 JOIN sales AS T2 ON T1.employee_id = T2.employee_id JOIN sale_details AS T3 ON T2.sale_id = T3.sale_id JOIN drug AS T4 ON T3.drug_id = T4.drug_id WHERE T2.total_amount IS NOT NULL AND T3.pharmacy_name LIKE 'A%' GROUP BY T1.employee_id;
**************************************************
{'schema': "    \n    CREATE TABLE supplier (\n    supplier_id INT PRIMARY KEY AUTO_INCREMENT, -- Unique ID for each supplier, auto-incremented\n    supplier_name VARCHAR(100) NOT NULL, -- Name of the supplier company or entity\n    contact_info VARCHAR(100), -- Contact details (e.g., phone, email) of the supplier\n    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP -- Date and time when the supplier record was created\n);\n\nCREATE TABLE drug_category (\n    category_id INT PRIMARY KEY AUTO_INCREMENT, -- Unique ID for each drug 

In [28]:
import ollama

# model_list = ['qwen2.5:7b','gemma3:4b','phi3','phi3.5']
model_list = ['llama3.2:3b']

dict=[]

for model in model_list:
    
    for i in range(10):
        FILENAME = f"table_list{i+1}_exp.sql"

        response = ollama.chat(model=model, messages=[
        {
            'role': 'user',
            # 'content': get_prompt("What are the names and prices of all drugs in the 'Antibiotics' category?")
            'content': get_custom_prompt(
                question = questions[i],
                sql_file = FILENAME
            )
            # 'content': get_prompt("Which pharmacies have sold more than 100 units of drugs from the 'Painkillers' category in the last 30 days, and what is the total revenue generated from those sales, including the names of the employees who made the sales?")
        },
        ])

        res = response['message']['content']
        

        responses.append(res)

        rating = get_ratings(
            schema = f"table_list{i+1}_exp.sql",
            question = questions[i],
            answer = res,
        )

        ratings.append(rating)

        d={
            'schema':read_file(FILENAME),
            'model':model,
            'question':questions[i],
            'sqlanswer':res,
            'rating':rating
        }

        dict.append(d)




In [29]:
print(responses[1])

SELECT e.first_name, e.last_name, d.department_name, s.store_name, SUM(sai.quantity) AS total_quantity FROM employee e JOIN store s ON e.store_id = s.store_id JOIN department d ON e.department_id = d.department_id JOIN product p ON s.id = p.category_id WHERE p.category_name IN ('TVs', 'Shirts') AND sales.sale_date >= '2022-01-01' GROUP BY e.first_name, e.last_name, d.department_name, s.store_name;


In [31]:
len(ratings)

20

In [33]:
for d in dict:
    print(f"{d['model']} --- {d['rating']}")

llama3.2:3b --- 9
llama3.2:3b --- 7
llama3.2:3b --- 8
llama3.2:3b --- 8
llama3.2:3b --- 9
llama3.2:3b --- 6
llama3.2:3b --- 6
llama3.2:3b --- 9
llama3.2:3b --- 9
llama3.2:3b --- 6


In [45]:
import pandas as pd

df = pd.DataFrame(dict)

In [46]:
df

,schema,model,question,sqlanswer,rating
0,\n CREATE TABLE supplier (\n supplie...,llama3.2:3b,List all drugs in the 'Painkillers' category w...,"SELECT d.drug_name, s.supplier_name, i.last_up...",9
1,CREATE TABLE store (\n store_id INT PRIMARY...,llama3.2:3b,Identify the top 5 products by total sales amo...,"SELECT T1.product_name, T3.category_name, T2.l...",7
2,CREATE TABLE user (\n user_id INT PRIMARY K...,llama3.2:3b,Find the top 3 products with the highest avera...,"SELECT product_name, \n c.category_name,...",8
3,CREATE TABLE agent (\n agent_id INT PRIMARY...,llama3.2:3b,List all agents who have facilitated sales of ...,"SELECT \n A.first_name,\n A.last_name,\n...",8
4,CREATE TABLE supplier (\n supplier_id INT P...,llama3.2:3b,Identify the top 5 products by total sales amo...,"SELECT p.product_name, e.first_name, COALESCE(...",9
5,CREATE TABLE restaurant (\n restaurant_id I...,llama3.2:3b,Retrieve the first name of customers who have ...,"SELECT c.first_name , r.name , COALESCE(SUM(od...",6
6,CREATE TABLE vehicle_type (\n type_id INT P...,llama3.2:3b,Retrieve the first and last name of customers ...,"SELECT T1.first_name, T1.last_name, T3.model, ...",6
7,CREATE TABLE customer (\n customer_id INT P...,llama3.2:3b,How can you retrieve a list of all active subs...,"SELECT c.first_name , p.plan_name , s.start_da...",9
8,CREATE TABLE warehouse (\n warehouse_id INT...,llama3.2:3b,How can you retrieve a report showing the tota...,"SELECT t1.product_name , SUM(t3.stock_level) ...",9
9,CREATE TABLE products (\n product_id IN...,llama3.2:3b,How can you generate a report showing the tota...,To answer this question with the given Postgre...,6


In [42]:
df.to_csv('result25.csv')

AttributeError: 'NoneType' object has no attribute 'to_csv'

In [43]:
df

In [44]:
# df_gemma = df[df['model']=='qwen2.5:7b']
# df_gemma